In [58]:
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipyfilechooser import FileChooser
import pandas as pd
from scipy import integrate
plt.ioff();

In [59]:
def init_fig():
    global fig
    fig = plt.figure()
    plt.close()
    

def add_scatter_xy():
    global fig
    global x,y
    plt.get_current_fig_manager().canvas.figure = fig
    plt.scatter(x,y)
    plt.show()

def add_scatter_y():
    global fig
    global y
    plt.get_current_fig_manager().canvas.figure = fig
    x = (np.ones_like(y.T)*range(y.shape[0])).T
    plt.scatter(x,y, s=0.1)
    plt.show()

def add_line_xy():
    global fig
    global x, y
    plt.get_current_fig_manager().canvas.figure = fig
    plt.plot(x, y)
    plt.show()

def add_line_y():
    global fig
    global y
    plt.get_current_fig_manager().canvas.figure = fig
    plt.plot(y)
    plt.show()

def show_fig():
    global fig
    plt.get_current_fig_manager().canvas.figure = fig
    plt.show()

def set_label_title():
    global xlabel, ylabel, title
    plt.get_current_fig_manager().canvas.figure = fig
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

def set_axis_lim():
    global fig
    global xlim_high, xlim_low
    global ylim_high, ylim_low
    plt.get_current_fig_manager().canvas.figure = fig
    plt.xlim([xlim_low, xlim_high])
    plt.ylim([ylim_low, ylim_high])
    plt.show()

def set_autoscale():
    global fig
    plt.get_current_fig_manager().canvas.figure = fig
    plt.gca().relim()
    plt.gca().autoscale()
    plt.show()

def eval_hd_variance():
    global param_T_0
    global param_C
    global param_delta_cep
    global param_T_p
    global param_S
    global param_hd_theta
    global param_hd_mode
    global hd_variance

    E_0 = 1

    integral_t = np.linspace(-5*param_T_0, 5*param_T_0, 1000)
    integral_y = (E_0*np.exp(-0.5*(integral_t/param_T_0)**2))**2 * (np.exp(- np.log(param_S)*np.exp(-0.5*(integral_t/param_T_p)**2)) * np.cos(param_hd_theta-param_delta_cep*(integral_t/param_T_0)-param_C/2.0*(integral_t/param_T_0)**2)**2 + np.exp(np.log(param_S)*np.exp(-0.5*(integral_t/param_T_p)**2)) * np.sin(param_hd_theta -param_delta_cep*(integral_t/param_T_0)- param_C/2.0*(integral_t/param_T_0)**2 )**2)

    numerator = integrate.simps(integral_y, integral_t)
    integral_y = (E_0*np.exp(-0.5*(integral_t/param_T_0)**2))**2
    denominator = integrate.simps(integral_y, integral_t)
    
    hd_variance = numerator / denominator

def sweep_time_variance():
    global param_T_0
    global param_C
    global param_delta_cep
    global param_T_p
    global param_S
    global hd_variance
    global param_hd_theta
    global x, y
    global xlabel
    global ylabel

    xlabel = 'time [a.u.]'
    ylabel = 'tempolal variance [dB]'

    x = np.linspace(-5*param_T_0, 5*param_T_0, 1000)
    integral_t = x
    y = (np.exp(- np.log(param_S)*np.exp(-0.5*(integral_t/param_T_p)**2)) * np.cos(param_hd_theta-param_delta_cep*(integral_t/param_T_0)-param_C/2.0*(integral_t/param_T_0)**2)**2 + np.exp(np.log(param_S)*np.exp(-0.5*(integral_t/param_T_p)**2)) * np.sin(param_hd_theta -param_delta_cep*(integral_t/param_T_0)- param_C/2.0*(integral_t/param_T_0)**2 )**2)
    y = 10*np.log10(y)

def sweep_time_LO():
    global param_T_0
    global param_delta_cep
    global param_C
    global param_hd_theta
    global x, y
    global xlabel
    global ylabel

    xlabel = 'time [a.u.]'
    ylabel = 'LO quad'

    E_0 = 1
    x = np.linspace(-5*param_T_0, 5*param_T_0, 1000)
    integral_t = x
    y = E_0*np.exp(-0.5*(integral_t/param_T_0)**2 + 1j*param_hd_theta -1j*param_delta_cep*integral_t/param_T_0  - 0.5j*param_C*(integral_t/param_T_0)**2)
    y = np.vstack([y.real,y.imag]).T

def sweep_T_0_squeeze():
    global param_T_0
    global hd_variance
    global x, y
    global xlabel
    global ylabel

    xlabel = 'T_0 [a.u.]'
    ylabel = 'variance [dB]'

    x = np.linspace(0.01,10,100)
    y = np.zeros_like(x)

    for i,j in enumerate(x):
        param_T_0 = j
        eval_hd_variance()
        y[i] = hd_variance
    y = 10*np.log10(y)

def sweep_T_p_squeeze():
    global param_T_p
    global hd_variance
    global x, y
    global xlabel
    global ylabel

    xlabel = 'T_p [a.u.]'
    ylabel = 'variance [dB]'

    x = np.linspace(0.01,10,100)
    y = np.zeros_like(x)

    for i,j in enumerate(x):
        param_T_p = j
        eval_hd_variance()
        y[i] = hd_variance
    y = 10*np.log10(y)

def sweep_T_0_ratio_squeeze():
    global param_T_0
    global param_T_p
    global hd_variance
    global x,y
    global xlabel
    global ylabel

    xlabel = 'T_0/T_p ratio'
    ylabel = 'variance [dB]'
    
    x = np.linspace(0.01,10,100)
    y = np.zeros_like(x)

    for i,j in enumerate(x):
        param_T_0 = j * param_T_p
        eval_hd_variance()
        y[i] = hd_variance
    y = 10*np.log10(y)




In [60]:
def simple_ui():
    button_clear_output = widgets.Button(description='表示クリア')
    button_init_fig = widgets.Button(description='初期化')
    button_set_label_title = widgets.Button(description='ラベル・タイトル')
    button_set_axis_lim = widgets.Button(description='軸範囲設定')
    button_set_autoscale = widgets.Button(description='自動範囲設定')
    button_add_line_xy = widgets.Button(description='プロット(x,y)')
    button_add_line_y = widgets.Button(description='プロット(y)')
    button_add_scatter_xy = widgets.Button(description='散布図(x,y)')
    button_add_scatter_y = widgets.Button(description='散布図(y)')
    button_show_fig = widgets.Button(description='描写')
    filechooser = FileChooser('./')
    button_load_as_x_npy = widgets.Button(description='x.npy読込')
    button_load_as_y_npy = widgets.Button(description='y.npy読込')
    button_load_as_x_txt = widgets.Button(description='x.txt読込')
    button_load_as_y_txt = widgets.Button(description='y.txt読込')
    button_load_csv_dialog   = widgets.Button(description='csv読込ダイアログ')
    button_input_field = widgets.Button(description='変数反映')

    text_xlabel = widgets.Text(value='',description='xlabel')
    text_ylabel = widgets.Text(value='',description='ylabel')
    text_title = widgets.Text(value='',description='title')
    text_xlim_high = widgets.Combobox(value='None',description='xlim_high',options=['None', '2020/1/1 00:00:00'],layout= widgets.Layout())
    text_xlim_low = widgets.Combobox(value='None',description='xlim_low',options=['None', '2019/1/1 00:00:00'],layout= widgets.Layout())
    text_ylim_high = widgets.Combobox(value='None',description='ylim_high',options=['None'],layout= widgets.Layout())
    text_ylim_low = widgets.Combobox(value='None',description='ylim_low',options=['None'],layout= widgets.Layout())
    text_delimiter = widgets.Combobox(value=',', description='delimiter',ensure_option=False, options=[',','\\t',' '])

    float_param_T_0 = widgets.FloatText(value=1,description='T_0')
    float_param_C = widgets.FloatText(value=0,description='C')
    float_param_delta_cep = widgets.FloatText(value=0,description='delta_cep')
    float_param_T_p = widgets.FloatText(value=1,description='T_p')
    float_param_S = widgets.FloatText(value=2,description='s')
    float_param_loss = widgets.FloatText(value=0, description='loss')
    float_param_hd_theta = widgets.FloatText(value=0, description='hd_theta')
    select_param_hd_mode = widgets.Dropdown(value='hd_theta', options=['hd_theta', 'min_var', 'max_var'], description='hd_mode')

    button_x_transpose = widgets.Button(description='x転置')
    button_y_transpose = widgets.Button(description='y転置')
    button_sweep_time_variance = widgets.Button(description='分散波形')
    button_sweep_time_LO = widgets.Button(description='LO波形')
    button_sweep_T_0_squeeze = widgets.Button(description='T_0スイープ')
    button_sweep_T_p_squeeze = widgets.Button(description='T_pスイープ')
    button_sweep_T_0_ratio_squeeze = widgets.Button(description='T_0/T_p比スイープ')
    


    output = widgets.Output()
    def wrapped_func_factory(func):
        def new_func(ui_element):
            with output:
                print(f"exec func {func.__name__}")
                func()
                print(f"complete {func.__name__}")
        return new_func
    button_clear_output.on_click(lambda button: output.clear_output(wait=False))
    button_init_fig.on_click(wrapped_func_factory(init_fig))
    button_set_label_title.on_click(wrapped_func_factory(set_label_title))
    button_add_line_xy.on_click(wrapped_func_factory(add_line_xy))
    button_add_line_y.on_click(wrapped_func_factory(add_line_y))
    button_add_scatter_xy.on_click(wrapped_func_factory(add_scatter_xy))
    button_add_scatter_y.on_click(wrapped_func_factory(add_scatter_y))
    button_show_fig.on_click(wrapped_func_factory(show_fig))
    button_set_axis_lim.on_click(wrapped_func_factory(set_axis_lim))
    button_set_autoscale.on_click(wrapped_func_factory(set_autoscale))
    button_sweep_time_variance.on_click(wrapped_func_factory(sweep_time_variance))
    button_sweep_time_LO.on_click(wrapped_func_factory(sweep_time_LO))
    button_sweep_T_0_squeeze.on_click(wrapped_func_factory(sweep_T_0_squeeze))
    button_sweep_T_p_squeeze.on_click(wrapped_func_factory(sweep_T_p_squeeze))
    button_sweep_T_0_ratio_squeeze.on_click(wrapped_func_factory(sweep_T_0_ratio_squeeze))

    def load_npy_factory(variable_name):
        def load_npy():
            choosed_file_path = filechooser.selected
            global x, y
            if variable_name == 'x':
                x = np.load(choosed_file_path)
                print(f'x.shape={x.shape}')
            elif variable_name == 'y':
                y = np.load(choosed_file_path)
                print(f'y.shape={y.shape}')
        return load_npy
    def load_txt_factory(variable_name):
        def load_txt():
            choosed_file_path = filechooser.selected
            global x, y
            if variable_name == 'x':
                x = np.loadtxt(choosed_file_path, delimiter=delimiter)
                print(f'x.shape={x.shape}')
            elif variable_name == 'y':
                y = np.loadtxt(choosed_file_path, delimiter=delimiter)
                print(f'y.shape={y.shape}')
        return load_txt
    def transpose_factory(variable_name):
        def transpose():
            global x,y
            if variable_name == 'x':
                x = x.T
                print(f'x.shape={x.shape}')
            elif variable_name == 'y':
                y = y.T
                print(f'y.shape={y.shape}')
        return transpose
    def load_csv_dialog():
        choosed_file_path = filechooser.selected
        global x,y
        df = pd.read_csv(choosed_file_path,delimiter=delimiter)
        selection = widgets.SelectMultiple(options=df.columns,description='columns')
        button_load_as_x = widgets.Button(description='x読込')
        button_load_as_y = widgets.Button(description='y読込')
        button_load_as_x_as_datetime = widgets.Button(description='x.datetime読込')
        def load_as_x():
            global x
            x = df[list(selection.value)].values
            print(f'x.shape={x.shape}')
        def load_as_y():
            global y
            y = df[list(selection.value)].values
            print(f'y.shape={y.shape}')
        def load_as_x_as_datetime():
            global x
            assert len(selection.value) == 1
            x = pd.to_datetime(df[selection.value[0]]).values
            print(f'x.shape={x.shape}')
        button_load_as_x.on_click(wrapped_func_factory(load_as_x))
        button_load_as_y.on_click(wrapped_func_factory(load_as_y))
        button_load_as_x_as_datetime.on_click(wrapped_func_factory(load_as_x_as_datetime))
        display(
            widgets.HBox([selection,button_load_as_x,button_load_as_y,button_load_as_x_as_datetime]),
            df)
        
    button_load_as_x_npy.on_click(wrapped_func_factory(load_npy_factory('x')))
    button_load_as_y_npy.on_click(wrapped_func_factory(load_npy_factory('y')))
    button_load_as_x_txt.on_click(wrapped_func_factory(load_txt_factory('x')))
    button_load_as_y_txt.on_click(wrapped_func_factory(load_txt_factory('y')))
    button_load_csv_dialog.on_click(wrapped_func_factory(load_csv_dialog))
    button_x_transpose.on_click(wrapped_func_factory(transpose_factory('x')))
    button_y_transpose.on_click(wrapped_func_factory(transpose_factory('y')))





    def load_input_field():
        global xlabel
        global ylabel
        global title
        global xlim_high, xlim_low
        global ylim_high, ylim_low
        global delimiter

        global param_T_0
        global param_C
        global param_delta_cep
        global param_T_p
        global param_S
        global param_loss
        global param_hd_theta
        global param_hd_mode
        param_T_0 = float_param_T_0.value
        param_C = float_param_C.value
        param_delta_cep = float_param_delta_cep.value
        param_T_p = float_param_T_p.value
        param_S = float_param_S.value
        param_loss = float_param_loss.value
        param_hd_theta = float_param_hd_theta.value
        param_hd_mode = select_param_hd_mode.value

        xlabel = text_xlabel.value
        ylabel = text_ylabel.value
        title = text_title.value
        lims = [text_xlim_high.value, text_xlim_low.value, text_ylim_high.value, text_ylim_low.value]
        def convert_lim(string):
            from dateutil import parser
            if string in ['None','']:
                return None
            else:
                try: return float(string)
                except: pass
                try: return parser.parse(string)
                except Exception as e: print(e)
        xlim_high, xlim_low, ylim_high, ylim_low = [convert_lim(i) for i in lims]
        delimiter = text_delimiter.value
    button_input_field.on_click(wrapped_func_factory(load_input_field))

    display(
        widgets.HBox([button_clear_output, button_init_fig, button_set_label_title ,button_set_axis_lim,button_set_autoscale , button_show_fig]),
        widgets.HBox([button_add_line_y,button_add_line_xy, button_add_scatter_y, button_add_scatter_xy]),
        widgets.HBox([filechooser,button_load_as_x_npy, button_load_as_y_npy]),
        widgets.HBox([button_load_as_x_txt,button_load_as_y_txt,button_load_csv_dialog]),
        widgets.Accordion(children=[
            widgets.VBox([
                widgets.HBox([text_xlabel, text_ylabel, text_title ,button_input_field]),
                widgets.HBox([text_xlim_low,text_xlim_high, text_ylim_low, text_ylim_high]),
                widgets.HBox([text_delimiter, float_param_T_p, float_param_S, float_param_hd_theta]),
                widgets.HBox([float_param_T_0, float_param_C, float_param_delta_cep, float_param_loss]),
                widgets.HBox([select_param_hd_mode])
            ])
        ],  _titles={0:'各種設定'}, selected_index=None),
        widgets.HBox([button_x_transpose,button_y_transpose, button_sweep_time_variance, button_sweep_time_LO,button_sweep_T_0_squeeze, button_sweep_T_p_squeeze]),
        widgets.HBox([button_sweep_T_0_ratio_squeeze, ]),
        output,)


In [61]:
simple_ui()

Accordion(children=(VBox(children=(HBox(children=(Text(value='', description='xlabel'), Text(value='', descrip…

Output()